# Resampling Methods Lab

In [182]:
import numpy as np  
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (
    ModelSpec as MS,
    summarize, 
    poly
)
from sklearn.model_selection import train_test_split    

In [183]:
from functools import partial
from sklearn.model_selection import (
    cross_validate,
    KFold,
    ShuffleSplit
)
from sklearn.base import clone
from ISLP.models import sklearn_sm 

## The Validation Set Approach

In [184]:
# Load the Auto dataset 
Auto = load_data('Auto')
Auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
name,,,,,,,,
chevrolet chevelle malibu,18.0,8,307.0,130,3504,12.0,70,1
buick skylark 320,15.0,8,350.0,165,3693,11.5,70,1
plymouth satellite,18.0,8,318.0,150,3436,11.0,70,1
amc rebel sst,16.0,8,304.0,150,3433,12.0,70,1
ford torino,17.0,8,302.0,140,3449,10.5,70,1


In [185]:
# Split the Auto dataset into training and validation sets
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=0)

In [186]:
# Fit a linear regression using only the observations corresponding to the training set
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

In [187]:
# Evaluate on test set using the predict() method of results
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
valid_mse = np.mean((y_valid - valid_pred) ** 2)
print(f"Validation MSE: {valid_mse:.2f}")

Validation MSE: 23.62


In [188]:
# Estimating the validation error for higher-degree polynomial regressions
def evalMSE(terms ,response ,train ,test):
    mm = MS(terms)
    X_train = mm. fit_transform (train)
    y_train = train[ response ]
    X_test = mm. transform (test)
    y_test = test[ response ]
    results = sm.OLS(y_train , X_train).fit ()
    test_pred = results.predict(X_test)
    return np.mean (( y_test - test_pred )**2)

In [189]:
# Evaluate MSE for polynomial degrees 1 to 3
MSE = np.zeros(3)
for idx , degree in enumerate (range (1, 4)):
    MSE[idx] = evalMSE (
        [poly('horsepower', degree)], 'mpg', Auto_train , Auto_valid
    )

print("Validation MSEs for polynomial degrees 1 to 3:\n", MSE)

Validation MSEs for polynomial degrees 1 to 3:
 [23.61661707 18.76303135 18.79694163]


In [190]:
# Re-evaluate with different random seed
Auto_train , Auto_valid = train_test_split (Auto, test_size=196, random_state =3)
MSE = np.zeros (3)
for idx , degree in enumerate (range (1, 4)):
    MSE[idx] = evalMSE ([ poly('horsepower', degree)], 'mpg', Auto_train , Auto_valid)

print("Validation MSEs for polynomial degrees 1 to 3:\n", MSE)

Validation MSEs for polynomial degrees 1 to 3:
 [20.75540796 16.94510676 16.97437833]


These results are consistent with our previous findings: a model that predicts mpg using a quadratic function of horsepower performs better than a model that involves only a linear function of horsepower, and there is no evidence of an improvement in using a cubic function of horsepower.

## Cross-Validation

The simplest way to cross-validate in Python is to use sklearn, which has a different interface or API than statsmodels, the code we have been using to fit GLMs. The ISLP wrapper package provides a wrapper, `sklearn_sm()`, that enables us to easily use `sklearn_sm()` the cross-validation tools of sklearn with models fit by statsmodels.  

The class `sklearn_sm()` has as its first argument a model from `statsmodels`. It can take two additional optional arguments: `model_str` which can be used to specify a formula, and `model_args` which should be a dictionary of additional arguments used when fitting the model.


In [191]:
# Leave-one-out cross-validation for linear regression model
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
X, Y = Auto.drop(columns =['mpg']), Auto['mpg']

cv_results = cross_validate(hp_model, X, Y, cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])

print("Validation MSE from leave-one-out cross-validation:\n", cv_err)

Validation MSE from leave-one-out cross-validation:
 24.231513517929226


In [192]:
# Leave-one-out cross-validation for polynomial regression models of degrees 1 to 5
cv_error = np.zeros (5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1, 6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M, X, Y, cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])

print("Validation MSEs from leave-one-out cross-validation for polynomial degrees 1 to 5:\n", cv_error)

Validation MSEs from leave-one-out cross-validation for polynomial degrees 1 to 5:
 [24.23151352 19.24821312 19.33498406 19.4244303  19.03321527]


In [193]:
# Example of np.add.outer
A = np.array ([3, 5, 9])
B = np.array ([2, 4])
np.add.outer(A, B)

array([[ 5,  7],
       [ 7,  9],
       [11, 13]])

In [194]:
# K-Fold Cross-Validation for polynomial regression
cv_error = np.zeros (5)
cv = KFold(n_splits=10, shuffle=True, random_state=0) # use same splits for each degree
for i, d in enumerate (range (1 ,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate (M, X, Y, cv=cv)
    cv_error [i] = np.mean(M_CV['test_score'])

print("cv_error:", cv_error)

cv_error: [24.20766449 19.18533142 19.27626666 19.47848402 19.13718373]


In [195]:
# Shuffle Split Cross-Validation for polynomial regression
validation = ShuffleSplit(n_splits=1, test_size=196, random_state=0)
results = cross_validate (hp_model, Auto.drop(['mpg'], axis =1), Auto['mpg'], cv=validation);

print("Shuffle Split CV test score:", results['test_score'])

Shuffle Split CV test score: [23.61661707]


In [196]:
# Shuffle Split Cross-Validation for polynomial regression
validation = ShuffleSplit(n_splits=10, test_size=196, random_state=0)
results = cross_validate (hp_model, Auto.drop(['mpg'], axis=1), Auto['mpg'], cv=validation)

print("Shuffle Split CV test score mean:", results['test_score'].mean())
print("Shuffle Split CV test score std:", results['test_score'].std())

Shuffle Split CV test score mean: 23.802232661034168
Shuffle Split CV test score std: 1.4218450941091916


## Bootsrapping

In [197]:
# Compute the optimal alpha for a given dataset and indices
Portfolio = load_data ('Portfolio')
Portfolio.head()

,X,Y
0,-0.895251,-0.234924
1,-1.562454,-0.885176
2,-0.417090,0.271888
3,1.044356,-0.734198
4,-0.315568,0.841983


In [198]:
# Compute the optimal alpha for a given dataset and indices
def alpha_func (D, idx):
    cov_ = np.cov(D[['X','Y']].iloc[idx], rowvar=False)
    return (( cov_ [1 ,1] - cov_ [0 ,1]) /
(cov_ [0 ,0]+ cov_ [1 ,1] -2* cov_ [0 ,1]))

In [199]:
alpha = alpha_func(Portfolio, range(100))
print("Optimal alpha:", alpha)

Optimal alpha: 0.57583207459283


In [200]:
# Compute the optimal alpha using bootstrap resampling
rng = np.random.default_rng(0)
alpha = alpha_func(Portfolio, rng.choice(100, 100, replace=True))

In [201]:
# Compute the bootstrap standard error for a given function
def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    # Bootstrap resampling
    for _ in range(B):
        # Generate bootstrap sample indices
        idx = rng.choice(D.index,n,replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value ** 2
    return np.sqrt(second_ / B - (first_ / B)**2)

In [202]:
# Shuffle Split Cross-Validation for polynomial regression
alpha_SE = boot_SE(alpha_func, Portfolio, B=1000, seed=0)
print("alpha_SE:", alpha_SE)

alpha_SE: 0.09118176521277699


### Estimating the Accuracy of a Linear Regression Model

In [203]:
# Bootstrap estimation of OLS regression coefficients
def boot_OLS (model_matrix , response , D, idx):
    try:
        D_ = D.iloc[idx]
    except:  
        D_ = D.loc[idx]
        
    Y_ = D_[response]
    # Transform the model matrix using the bootstrap sample
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_ , X_).fit().params

In [204]:
# Bootstrap estimation of OLS regression coefficients
hp_func = partial(boot_OLS , MS(['horsepower']), 'mpg')

In [205]:
hp_func?

Signature:      hp_func(D, idx)
Call signature: hp_func(*args, **kwargs)
Type:           partial
String form:    functools.partial(<function boot_OLS at 0x15d370d50>, ModelSpec(terms=['horsepower']), 'mpg')
File:           /opt/miniconda3/envs/islp/lib/python3.14/functools.py
Docstring:     
Create a new function with partial application of the given arguments
and keywords.

In [ ]:
# Bootstrap estimation of OLS regression coefficients demonstration with 10 bootstrap samples
rng = np.random.default_rng(0)
np.array(
    [
        hp_func(
            Auto, 
            rng.choice(392, 392, replace=True)
        ) for _ in range(10) 
    ]
)

array([[39.88064456, -0.1567849 ],
       [38.73298691, -0.14699495],
       [38.31734657, -0.14442683],
       [39.91446826, -0.15782234],
       [39.43349349, -0.15072702],
       [40.36629857, -0.15912217],
       [39.62334517, -0.15449117],
       [39.0580588 , -0.14952908],
       [38.66688437, -0.14521037],
       [39.64280792, -0.15555698]])

In [ ]:
# Bootstrap estimation of OLS regression coefficients standard error with 1000 bootstrap samples
hp_se = boot_SE(hp_func, Auto, B=1000, seed=10)

print("Horsepower standard error:\n", hp_se)

Horsepower standard error:
 intercept     0.731176
horsepower    0.006092
dtype: float64


In [ ]:
# Bootstrap estimation of OLS regression coefficients using the hp_model defined earlier
hp_model.fit(Auto , Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

In [212]:
# Shuffle Split Cross-Validation for polynomial regression
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS, quad_model, 'mpg')
boot_SE(quad_func, Auto, B=1000)

intercept                                  1.538641
poly(horsepower, degree=2, raw=True)[0]    0.024696
poly(horsepower, degree=2, raw=True)[1]    0.000090
dtype: float64

In [213]:
M = sm.OLS(Auto['mpg'],
quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64